In [35]:
import pandas as pd
import boto3
import json

In [36]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

,Param,Value
0,DWH_CLUSTER_TYPE,multi-node
1,DWH_NUM_NODES,4
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,dwh-cluster
4,DWH_DB,dwh
5,DWH_DB_USER,dwhuser
6,DWH_DB_PASSWORD,PASSw0rd
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,dwhRole


In [37]:
import boto3

ec2 = boto3.resource('ec2',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )

s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )

iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-west-2'
                  )

redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

## Check out the sample data sources on S3

In [38]:
sampleDbBucket =  s3.Bucket("udacity-dend")
for obj in sampleDbBucket.objects.filter(Prefix="log_data"):
    print(obj)
#for obj in sampleDbBucket.objects.all():
 #   print(obj)

s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-01-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-02-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-03-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-04-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-05-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-06-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-07-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-08-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-09-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-10-events.json')
s3.ObjectSummary(b

## STEP 1: IAM ROLE
.Create an IAM Role that makes Redshift able to access S3 bucket (ReadOnly)

In [39]:
from botocore.exceptions import ClientError

#1.1 Create the role, 
try:
    print("1.1 Creating a new IAM Role") 
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )    
except Exception as e:
    print(e)
    
    
print("1.2 Attaching Policy")

iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

print("1.3 Get the IAM role ARN")
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

## STEP 2: Redshift Cluster
Create a RedShift Cluster

In [40]:
try:
    response = redshift.create_cluster(        
        #HW
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        #Identifiers & Credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles=[roleArn]  
    )
except Exception as e:
    print(e)

## 2.1 *Describe* the cluster to see its status
- run this block several times until the cluster status becomes `Available`

In [47]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

,Key,Value
0,ClusterIdentifier,dwh-cluster
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,dwhuser
4,DBName,dwh
5,Endpoint,"{'Address': '', 'Port': }"
6,VpcId,vpc-22760e5a
7,NumberOfNodes,4


## 2.2 Take note of the cluster endpoint and role ARN 
DO NOT RUN THIS unless the cluster status becomes "Available"

In [48]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

DWH_ENDPOINT :: 
DWH_ROLE_ARN ::  


## STEP 3: Open an incoming TCP port to access the cluster ednpoint¶

In [51]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

ec2.SecurityGroup(id='sg-73915a25')
An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: 0.0.0.0/0, TCP, from port: 5439, to port: 5439, ALLOW" already exists


## STEP 4: Make sure you can connect to the cluster

In [52]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [53]:
conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
print(conn_string)
%sql $conn_string

## STEP 5: Clean up your resources

<b><font color='red'>DO NOT RUN THIS UNLESS YOU ARE SURE <br/> 
    We will be using these resources in the next exercises</span></b>

In [70]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
#### CAREFUL!!

- run this block several times until the cluster really deleted

In [69]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

ClusterNotFoundFault: An error occurred (ClusterNotFound) when calling the DescribeClusters operation: Cluster dwh-cluster not found.

In [67]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)
#### CAREFUL!!

{'ResponseMetadata': {'RequestId': '58051cce-b5a9-4a9d-9941-a18147a31303',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '58051cce-b5a9-4a9d-9941-a18147a31303',
   'content-type': 'text/xml',
   'content-length': '200',
   'date': 'Thu, 06 Aug 2020 10:17:02 GMT'},
  'RetryAttempts': 0}}

In [57]:
%sql SELECT * FROM users LIMIT 5;

 
5 rows affected.


user_id,first_name,last_name,gender,level
95,Sara,Johnson,F,paid
57,Katherine,Gay,F,free
91,Jayden,Bell,M,free
27,Carlos,Carter,M,free
78,Chloe,Roth,F,free


In [59]:
%sql SELECT * FROM artists LIMIT 5;

 *
5 rows affected.


artist_id,name,location,lattitude,longitude
ARXDGV71187B990B08,Super Chikan,"Darling, MS",34,-90
AREQD801187B98AA22,George Ducas,,None,None
ARASYMJ1187B9ACAF2,Whitesnake,"London, England",51,0
ARBKGAY1187B98DEBE,Greta Y Los Garbo,,None,None
ARYFZUZ11F4C8438AD,Avengers,,None,None


In [60]:
%sql SELECT * FROM songplay LIMIT 5;

 * 
5 rows affected.


songplay_id,start_time,user_id,level,song_id,artist_id,session_id,location,user_agent
45,2018-11-12 11:45:25.796000,26,free,SOBONKR12A58A7A7E0,AR5E44Z1187B9A1D74,491,"Pikeville, KY","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"""
109,2018-11-15 13:44:58.796000,30,paid,SOBONKR12A58A7A7E0,AR5E44Z1187B9A1D74,324,"Pikeville, KY",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0
173,2018-11-30 04:57:03.796000,49,paid,SOEMXXF12A6D4F932C,ARI4S0E1187B9B06C0,1079,Tennessee,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20100101 Firefox/31.0
237,2018-11-28 22:32:42.796000,73,paid,SOBONKR12A58A7A7E0,AR5E44Z1187B9A1D74,954,"Pikeville, KY","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"""
301,2018-11-02 18:02:42.796000,50,free,SOBONKR12A58A7A7E0,AR5E44Z1187B9A1D74,207,"Pikeville, KY","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"""


In [61]:
%sql SELECT * FROM staging_events LIMIT 5;

 *
5 rows affected.


ste_artist,ste_auth,ste_fname,ste_gender,ste_itemsinsessions,ste_lname,ste_length,ste_level,ste_location,ste_method,ste_page,ste_registration,ste_session_id,ste_song,ste_status,ste_ts,ste_useragent,ste_user_id
None,Logged Out,None,None,0,None,None,free,None,PUT,Login,None,52,None,307,1541207073796,None,None
None,Logged In,Celeste,F,1,Williams,None,free,"Klamath Falls, OR",GET,Home,1541077528796,52,None,200,1541207123796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36""",53
Mynt,Logged In,Celeste,F,2,Williams,166,free,"Klamath Falls, OR",PUT,NextSong,1541077528796,52,Playa Haters,200,1541207150796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36""",53
Taylor Swift,Logged In,Celeste,F,3,Williams,230,free,"Klamath Falls, OR",PUT,NextSong,1541077528796,52,You Belong With Me,200,1541207316796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36""",53
Amy Winehouse,Logged In,Celeste,F,4,Williams,229,free,"Klamath Falls, OR",PUT,NextSong,1541077528796,52,Valerie,200,1541207546796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36""",53


In [62]:
%sql SELECT * FROM staging_songs LIMIT 5;

 * 
5 rows affected.


num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
1,ARC43071187B990240,None,None,"Wisner, LA",Wayne Watson,SOKEJEJ12A8C13E0D0,The Urgency (LP Version),245,0
1,AR1WWVL1187B9B0306,None,None,"Birmingham, England",UB40,SOZODBG12A6701C5D1,Reasons (Radio Remix),207,0
1,ARGQJWL1187FB3CE9E,None,None,,La Polla Records,SOXLGTI12A8C13C08B,Cara Al Culo,82,1985
1,ARR9XGB1187B9912E6,None,None,"Philadelphia, PA",Boyz II Men,SOFRAZJ12A8C143CDB,On Bended Knee,329,1994
1,ARAEX7Y1187FB42D36,None,None,"Severn, MD",Toni Braxton Featuring Shaggy,SOEXSIN12A8C133B0A,Christmas In Jamaica,219,2001


In [63]:
%sql SELECT * FROM songs LIMIT 5;

 * 
5 rows affected.


song_id,title,artist_id,year,duration
SOGKKYT12AB018185F,Friend Bob Marley,AR2ZLR21187B98D552,1997,163
SOILTBC12AB018483E,Little Mustang,AR8X9IT1187FB44B6D,0,262
SOLLNII12AB017BBFB,Seaside,AR6OLCY1187FB4D0C0,2003,68
SOZBDRT12A8C139A4B,Supercar Trip Album Version,ARCM3XF1187B9A9A0C,2005,275
SOERICN12AF72A4077,Frangenti,ARPOBPN1187B9ADA45,1999,231


In [64]:
%sql SELECT * FROM time LIMIT 5;

 * 
5 rows affected.


start_time,ts,hour,day,week,month,year,weekday
2018-11-03 01:12:26.796000,1541207546796,1,3,44,11,2018,6
2018-11-03 15:59:38.796000,1541260778796,15,3,44,11,2018,6
2018-11-03 16:34:03.796000,1541262843796,16,3,44,11,2018,6
2018-11-03 16:54:02.796000,1541264042796,16,3,44,11,2018,6
2018-11-03 17:10:11.796000,1541265011796,17,3,44,11,2018,6
